In [1]:
# Dependencies
from bs4 import BeautifulSoup as soup
from splinter import Browser
import requests
import pymongo
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Lunos\.wdm\drivers\chromedriver\win32\90.0.4430.24]


In [3]:
#Define a function to perform the repeated calls for 
def retrievehtml(url, target):
    browser.visit(url)
    html = browser.html
    html_soup = soup(html, 'html.parser')
    return html_soup.select_one(target)

In [4]:
# URL of first page to be scraped
url = 'https://redplanetscience.com/'
slide_elem = retrievehtml(url, 'div.list_text')

In [5]:
news_title = slide_elem.find('div', class_='content_title').get_text()
print(news_title)
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
print(news_p)

NASA's Mars Perseverance Rover Passes Flight Readiness Review
​The agency's Mars 2020 mission has one more big prelaunch review – the Launch Readiness Review, on July 27.


In [6]:
#Finding the Featured Image of the Day for https://spaceimages-mars.com
url2 = 'https://spaceimages-mars.com'
mars_pic_elem = retrievehtml(url2, 'div.header')
img = mars_pic_elem.find('img', class_='headerimage fade-in')
featured_image_url=(f"{url2}/{img['src']}")
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [7]:
#Get the URL of the full images from Marshemispheres.com
url3 = 'https://marshemispheres.com/'
astropedia_elem = retrievehtml(url3, 'div.result-list')
products = astropedia_elem.find_all('a', class_='itemLink')
subpages = []
for product in products:
    productURL = f"{url3}{product['href']}"
    if (productURL not in subpages):
        subpages.append(productURL)
subpages

['https://marshemispheres.com/cerberus.html',
 'https://marshemispheres.com/schiaparelli.html',
 'https://marshemispheres.com/syrtis.html',
 'https://marshemispheres.com/valles.html']

In [8]:
hemispheres = []
for page in subpages:
    hspage = retrievehtml(page,'div.cover')
    pagetitle = hspage.h2.get_text()
    hspage = retrievehtml(page,'div.container')
    imgurl = hspage.find('img', class_="wide-image")['src']
    hemispheres.append({"Title": pagetitle ,"Image URL" : imgurl})
hemispheres

[{'Title': 'Cerberus Hemisphere Enhanced',
  'Image URL': 'images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'Title': 'Schiaparelli Hemisphere Enhanced',
  'Image URL': 'images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'Image URL': 'images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'Image URL': 'images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]